# Tutorial for HP1alpha dimer slab simulation.

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import MOFFParser
from openabc.forcefields import MOFFMRGModel
import openabc.utils.helper_functions as helper_functions
from openabc.utils.insert import insert_molecules
from openabc.utils.CA2AA import multiple_chains_CA2AA

# set simulation platform
#platform_name = 'CPU'
platform_name = 'CUDA'

## 1. Build a single HP1alpha dimer

Following steps in single_hp1alpha_dimer.ipynb, we first build a single hp1alpha dimer by processing topological information from an initial input PDB file.

In [2]:
hp1alpha_dimer_parser = MOFFParser.from_atomistic_pdb('input-pdb/hp1a.pdb', 'hp1alpha_dimer_CA.pdb')

old_native_pairs = hp1alpha_dimer_parser.native_pairs.copy()
new_native_pairs = pd.DataFrame(columns=old_native_pairs.columns)
cd1 = np.arange(16, 72)
csd1 = np.arange(114, 176)
n_atoms_per_hp1alpha_dimer = len(hp1alpha_dimer_parser.atoms.index)
print(f'There are {n_atoms_per_hp1alpha_dimer} CA atoms in each HP1alpha dimer.')
cd2 = cd1 + int(n_atoms_per_hp1alpha_dimer/2)
csd2 = csd1 + int(n_atoms_per_hp1alpha_dimer/2)
for i, row in old_native_pairs.iterrows():
    a1, a2 = int(row['a1']), int(row['a2'])
    if a1 > a2:
        a1, a2 = a2, a1
    flag1 = ((a1 in cd1) and (a2 in cd1)) or ((a1 in csd1) and (a2 in csd1))
    flag2 = ((a1 in cd2) and (a2 in cd2)) or ((a1 in csd2) and (a2 in csd2))
    flag3 = ((a1 in csd1) and (a2 in csd2))
    if flag1 or flag2 or flag3:
        new_native_pairs.loc[len(new_native_pairs.index)] = row
hp1alpha_dimer_parser.native_pairs = new_native_pairs
hp1alpha_dimer_parser.parse_exclusions() # update exclusions based on the new native pairs

Parse molecule with default settings.
Get native pairs with shadow algorithm.
There are 382 CA atoms in each HP1alpha dimer.


## 2. Setup and perform slab simulations of 20 HP1alpha dimers.

We now show the steps for setting up and performing slab simulations. To save computational time, we only included 20 HP1alpha dimers in the system. To study phase behaviors and avoid finite-size effects, you should use more molecules (~100).

First, we randomly place 20 dimers in a cubic box of 50x50x50 nm^3 in size. 

In [3]:
n_mol = 100
if not os.path.exists('start.pdb'):
    insert_molecules('hp1alpha_dimer_CA.pdb', 'start.pdb', n_mol, box=[50, 50, 50])
else:
    # delete and create a new start.pdb
    # otherwise, there will be an error for init_coord in the next step
    os.remove('start.pdb')
    insert_molecules('hp1alpha_dimer_CA.pdb', 'start.pdb', n_mol, box=[50, 50, 50])

Successfully inserted 100 molecules.


In [4]:
# visualize start.pdb
pdb = mdtraj.load_pdb('start.pdb')
print('Show start.pdb structure.')
view = nglview.show_mdtraj(pdb)
view

Show start.pdb structure.


NGLWidget()

Next, we compress the system with an NPT simulation to create a dense protein-phase. To save computational time, we set the simulation steps as 200 in the Notebook. However, 2,000,000 steps is recommended to fully compress the system 

Note the NPT compression is performed under 1 bar and 150 K. However, the parameters for electrostatic interactions are defined based on 300 K. Since the compression is only used for producing an initial configuration, this inconsistency does not impact results of further slab simulations.

In [5]:
multi_dimers = MOFFMRGModel()
for i in range(n_mol):
    # append multiple hp1alpha dimer parser instances
    multi_dimers.append_mol(hp1alpha_dimer_parser)
multi_dimers.native_pairs.loc[:, 'epsilon'] = 6.0

box_a, box_b, box_c = 50, 50, 50
top = app.PDBFile('start.pdb').getTopology()
multi_dimers.create_system(top, box_a=box_a, box_b=box_b, box_c=box_c)
init_coord = app.PDBFile('start.pdb').getPositions()
salt_conc = 82*unit.millimolar

multi_dimers.add_protein_bonds(force_group=1)
multi_dimers.add_protein_angles(force_group=2, verbose=False)
multi_dimers.add_protein_dihedrals(force_group=3)
multi_dimers.add_native_pairs(force_group=4)
multi_dimers.add_contacts(force_group=5)
multi_dimers.add_elec_switch(salt_conc, temperature=300*unit.kelvin, force_group=6)

# follow the example provided by OpenMM user guide, use LangevinMiddleIntegrator and MonteCarloBarostat to perform NPT simulation
pressure = 1*unit.bar
temperature = 150*unit.kelvin
multi_dimers.system.addForce(mm.MonteCarloBarostat(pressure, temperature))
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
multi_dimers.set_simulation(integrator, platform_name, init_coord=init_coord)
multi_dimers.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output_NPT.dcd'
multi_dimers.add_reporters(output_interval, output_dcd)
multi_dimers.simulation.context.setVelocitiesToTemperature(temperature)
multi_dimers.simulation.step(100000) # only run 200 steps as an example, in principle we need more steps to compress the system
    

Add protein bonds.
Add protein dihedrals.
Add native pairs.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Add electrostatic interactions between native pair atoms.
Use platform: CUDA
Use precision: mixed
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,-276320.49920404627,71014.79840916084,-205305.70079488543,149.0635365245898,0
2000,20.000000000000327,-275301.7662525792,71773.4461555867,-203528.32009699254,150.65597526400697,2.24e+03
3000,30.00000000000189,-275466.7678479987,71459.7645029087,-204007.00334509002,149.99754212699156,2.17e+03
4000,40.00000000000061,-274752.13327913935,71507.86263690854,-203244.2706422308,150.09850246363777,2.17e+03
5000,49.99999999999862,-274589.40006095567,71968.17828285872,-202621.22177809695,151.06472752714626,2.13e+03
6000,59.99999999999663,-274706.62645205

From the compressed configuration, we can perform slab simulations at constant temperature and volume by fixing the box length at x and y direction, while extending long the z-axis. For example, the NPT simulation produced a configuration of approximately 15x15x15 nm^3 in size. Therefore, we set the slab simulation box size as 15x15x200 nm^3. The z-axis length was chosen to be large enough to create a liquid-vapor interface. In the meantime, too large a value may slow down the exchange of protein molecules between dense and dilute phases, hindering equilibration. Therefore, the box size is a system-dependent parameter that needs to be adjusted based on results from the NPT simulation.

In the following example, we did not use the configuration produced from the above NPT simulation, which is too short to fully compress the system. Instead, we used a trajectory produced from a much longer simulation: "NPT-output-files/NPT_compress.dcd". 

In [ ]:
# set slab simulation box size
#box_a = 15
#box_b = 15
#box_c = 200

box_a = 25
box_b = 25
box_c = 25

# load trajectory and get the compressed configuration
# for easier visualization, we move the geometric center of all the atoms to the box center
#npt_traj = mdtraj.load_dcd('NPT-output-files/NPT_compress.dcd', top='start.pdb')
npt_traj = mdtraj.load_dcd('output_NPT.dcd', top='start.pdb')
init_coord = npt_traj.xyz[-1]
init_coord -= np.mean(init_coord, axis=0)
init_coord += 0.5*np.array([box_a, box_b, box_c])

# we have to rebuild the system as this time there is no MonteCarloBarostat in it
multi_dimers = MOFFMRGModel()

for i in range(n_mol):
    # append multiple hp1alpha dimer parser instances
    multi_dimers.append_mol(hp1alpha_dimer_parser)

multi_dimers.native_pairs.loc[:, 'epsilon'] = 6.0

top = app.PDBFile('start.pdb').getTopology()
multi_dimers.create_system(top, box_a=box_a, box_b=box_b, box_c=box_c)
salt_conc = 82*unit.millimolar
temperature = 300*unit.kelvin
multi_dimers.add_protein_bonds(force_group=1)
multi_dimers.add_protein_angles(force_group=2, verbose=False)
multi_dimers.add_protein_dihedrals(force_group=3)
multi_dimers.add_native_pairs(force_group=4)
multi_dimers.add_contacts(force_group=5)
multi_dimers.add_elec_switch(salt_conc, temperature, force_group=6)

# use Nose-Hoover integrator to accelerate the dynamics
collision = 1/unit.picosecond
timestep = 5*unit.femtosecond
integrator = mm.NoseHooverIntegrator(temperature, collision, timestep)
multi_dimers.set_simulation(integrator, platform_name, init_coord=init_coord)
multi_dimers.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output_slab.dcd'
multi_dimers.add_reporters(output_interval, output_dcd)
multi_dimers.simulation.context.setVelocitiesToTemperature(temperature)
multi_dimers.simulation.step(1000000)

Add protein bonds.
Add protein dihedrals.
Add native pairs.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Add electrostatic interactions between native pair atoms.
Use platform: CUDA
Use precision: mixed
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,4.999999999999916,-139961.10645063865,114347.61177367621,-25613.494676962437,240.0212319961497,0
2000,10.000000000000163,-117495.85762659174,146645.8161983306,29149.95857173887,307.8167433935611,392
3000,15.000000000000945,-124966.56127103555,144187.0209068308,19220.45963579524,302.6556117709768,401
4000,20.000000000000306,-130372.58192103298,142073.79392611422,11701.212005081237,298.21984494094437,404
5000,24.99999999999931,-135639.35600026185,142139.1277320985,6499.771731836663,298.3569837964055,409
6000,29.999999999998316,-139805.97139261005,143792.394645070

## 3. Convert coarse-grained to all-atom configurations

To facilitate explicit-solvent atomistic simulations of condensate, we provide tools for converting the coarse-grained structures to atomistic ones. 

The conversion uses the software, REMO, which can be downloaded from [REMO](https://zhanggroup.org/REMO/REMO.v3.tar.bz2). By default, we installed the software in openabc/utils folder. If you put it in other places, please specify the REMO path in `multiple_chains_CA2AA` with the parameter `REMO_path`. REMO is not included in this GitHub repository due to copyright reasons. 

In [ ]:
run_convert = False # set to True if you want to run this part, which is time consuming

if run_convert:
    # start from the final snapshot of the previous short slab NVT trajectory
    state = multi_dimers.simulation.context.getState(getPositions=True, enforcePeriodicBox=True)
    positions = np.array(state.getPositions().value_in_unit(unit.nanometer))

    # write CA pdb with the target positions
    df_atoms = helper_functions.parse_pdb('start.pdb')
    df_atoms.loc[:, ['x', 'y', 'z']] = positions*10 # convert nm to A
    helper_functions.write_pdb(df_atoms, 'slab_NVT_relaxed.pdb')

    # convert CA pdb to AA pdb
    # note each molecule has 2 monomers, and each monomer has 191 residues
    # thus there are 2*n_mol chains, and each chain has 191 residues
    multiple_chains_CA2AA('slab_NVT_relaxed.pdb', [2*n_mol], [191])
    
    # show all-aton structure
    pdb = mdtraj.load_pdb('slab_NVT_relaxed_AA.pdb')
    print('Show HP1alpha dimer condensate all-atom structure.')
    view = nglview.show_mdtraj(pdb)
    view

After conversion, you can run all-atom simulations with your favorite software. 